In [1]:
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
from matplotlib.widgets import Cursor
import matplotlib.colors as mcolors
import matplotlib.animation as animation
from matplotlib.path import Path
from ipywidgets import interact # interactive plots in Jupyter Notebook
import pyidi                    # Python HSC data analysis library
# from scipy.interpolate import make_interp_spline, BSpline
import pickle as pk
import os
from pixel_setter2 import PixelSetter#, play_video, detect_peaks
from skimage import feature, measure
# from skimage.morphology import skeletonize
from skimage.feature import match_template
from mpl_toolkits import mplot3d
import dill

from scipy.signal import correlate2d
%matplotlib qt

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


Open the file to analyze and set an analysis method (Lucas-Kanade implementation)

In [2]:
# file_name = 'Single_line_15_S01'
# del file_path
folder_paths = ['H:/My Drive/PHD/HSC', 'D:/HSC', 'F:/', 'E:/thijs/', 'C:/Users/thijs/Documents/HSC/', 'D:/thijsmas/HSC']

# file_name = "Full_web_ecc0_new_Floc4_v1"
file_name = "Full_web_ecc0_new2_Floc15_v0"
file_name_video = file_name + "_S01.cihx"

# Search for the file in the folders and their subdirectories
for folder_path in folder_paths:
    for root, dirs, files in os.walk(folder_path, topdown=False):
        if file_name_video in files:
            file_path = os.path.join(root, file_name_video)
            print(file_path)
            break

# video_path = path + file_name_video +'/' + file_name_video+'.cihx'
video = pyidi.pyIDI(file_path)
# video_path = "C:/Users/thijs/Documents/GitHub/pyidi/data/"
# file_name = 'synthetic_data_thijs_without_rotation'
# video = pyidi.pyIDI(video_path + file_name +'.cih')
# video.set_method('sof')
mraw = video.mraw
video.info

# ani = play_video(video, range(400,1200), interval=30)
# writer = animation.FFMpegWriter(fps=60)
# ani.save('anim/'+file_name + '_anim.mp4', writer=writer)
# print('animation generated')

# data_dir = r'C:\Users\thijs\Documents\GitHub\LDAQ\examples\Data'
# measurement_path = os.path.join(data_dir, file_name+'.pkl')
# with open(measurement_path, 'rb') as f:
#     measurement = pk.load(f)


# file_name = 'Full_web12_S01'

H:/My Drive/PHD/HSC\ecc0\web1\Full_web_ecc0_new2_Floc15_v0_S01\Full_web_ecc0_new2_Floc15_v0_S01.cihx
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc15_v0_S01\Full_web_ecc0_new2_Floc15_v0_S01.cihx


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


{'Date': '2023/9/20',
 'Camera Type': 'FASTCAM SA-Z type 2100K-M-64GB',
 'Record Rate(fps)': 8000.0,
 'Shutter Speed(s)': 8104.0,
 'Total Frame': 8000,
 'Original Total Frame': 87371,
 'Image Width': 1024,
 'Image Height': 512,
 'File Format': 'Mraw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Higher',
 'Color Bit': 16,
 'Comment Text': ''}

Plot a frame in the video:

In [32]:
%matplotlib qt
sequential_image_n = 0
still_image = mraw[sequential_image_n]
still_image1 = mraw[sequential_image_n+1]

sigma_vec = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])*3
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(10, 5))
for ax_i, sigma in enumerate(sigma_vec):
    row = ax_i // 3
    col = ax_i % 3
    edges = feature.canny(still_image, sigma=sigma)
    ax[row, col].imshow((2**16-1)-edges, cmap='gray')
    # skeleton = skeletonize(edges)
    # contours = measure.find_contours(edges, sigma)
    # ax[row, col].imshow(still_image, cmap='gray')
    # for contour in contours:
        # ax[row, col].plot(contour[:, 1], contour[:, 0], linewidth=2)


In [4]:
pix_set = PixelSetter(still_image, file_name = file_name)
# pix_set.get_tracking_pixels()
pix_set.choose_reference_centers()


In [11]:
pix_set.combine_tracking_points()
pix_set2 = PixelSetter(still_image1, file_name = file_name)
for reference_image_center_i, reference_image_center_j in zip(pix_set.reference_image_center_i, pix_set.reference_image_center_j):
    pix_set2.add_reference_image(reference_image_center_i, reference_image_center_j)

In [12]:
pix_set2.combine_tracking_points()
pix_set.tracking_points

[(266, 477), (237, 115), (337, 433), (245, 518), (502, 69)]

In [13]:

fig, ax = plt.subplots(ncols=3, nrows=len(pix_set.ref_imgs), figsize=(10, 10))
fig.tight_layout()
for ax_i, img in enumerate(pix_set.ref_imgs):
    ax[ax_i,0 ].imshow(img, cmap='gray', vmin=0, vmax=2**16-1)
# for ax_i, img in enumerate(pix_set2.ref_imgs):
    # ax[ax_i,1 ].imshow(img, cmap='gray', vmin=0, vmax=2**16-1)
scale = 0.3
lim = 60000
X, Y = np.meshgrid(np.arange(0, pix_set.ref_imgs[0].shape[1]), np.arange(0, pix_set.ref_imgs[0].shape[0]))
for ax_i, img in enumerate(pix_set.ref_imgs):
    ax[ax_i,1].set_aspect('equal')
    ax[ax_i,1].set_ylim(img.shape[0]-.5, -.5)
    ax[ax_i,1].set_xlim(-.5,img.shape[0]-.5)
    grad = np.gradient(img)

    ax[ax_i,1 ].quiver(X, Y, grad[1], -grad[0], scale=300000)
    At   = np.array([grad[1].flatten(), grad[0].flatten()])
    A    = At.T
    eig_vals, eig_vecs = np.linalg.eigh(A.T @ A)
    print(np.linalg.inv(A.T @ A))
    print(eig_vals)
    ax[ax_i,2].plot(grad[1], grad[0],'k.')
    ax[ax_i,2].plot([0, -scale*eig_vals[0]**0.5*eig_vecs[0,0]], [0, -scale*eig_vals[0]**0.5*eig_vecs[0,1]],'r')
    ax[ax_i,2].plot([0, -scale*eig_vals[1]**0.5*eig_vecs[1,0]], [0, -scale*eig_vals[1]**0.5*eig_vecs[1,1]],'r')
    ax[ax_i,2].set_aspect('equal')
    ax[ax_i,2].set_xlim(-lim, lim)
    ax[ax_i,2].set_ylim(lim, -lim)
    ax[ax_i,2].set_xlabel('Ix')
    ax[ax_i,2].set_ylabel('Iy')

fig, ax = plt.subplots(ncols=2, nrows=len(pix_set.ref_imgs), figsize=(10, 10))
fig.tight_layout()
for ax_i, img in enumerate(zip(pix_set.ref_imgs, pix_set2.ref_imgs)):
    ax[ax_i,0 ].imshow(img[0], cmap='gray', vmin=0, vmax=2**16-1)
    print(np.max(img[0].astype(np.int32) - img[1].astype(np.int32)))
    vmax = 2600
    ax[ax_i,1 ].imshow(vmax - (img[0].astype(np.int32) - img[1].astype(np.int32)), cmap='gray', vmin=0, vmax=2600)

fig, ax = plt.subplots()
ax.imshow(pix_set.image, cmap='gray', vmin=0, vmax=2**16-1)
ax.plot(pix_set.reference_image_center_j, pix_set.reference_image_center_i, 'r.')

[[4.82138860e-09 9.90002291e-10]
 [9.90002291e-10 8.59709618e-09]]
[1.13110259e+08 2.18457293e+08]
[[2.38881981e-09 6.16174572e-10]
 [6.16174572e-10 2.52395262e-10]]
[3.91574280e+08 1.14388743e+10]
[[ 2.03628580e-10 -6.43576743e-11]
 [-6.43576743e-11  9.07012586e-11]]
[4.29588951e+09 1.62471708e+10]
[[ 2.91977691e-10 -2.84082744e-11]
 [-2.84082744e-11  5.51928408e-10]]
[1.80181238e+09 3.46129322e+09]
[[ 1.28423084e-10 -9.40751913e-13]
 [-9.40751913e-13  4.55021267e-11]]
[7.78611506e+09 2.19821502e+10]
704
3136
2272
800
1616


In [ ]:
pix_set.save(file_name)

In [ ]:
pix_set.save(path='C:/Users/thijsmas/Documents/GitHub/pyidi_data/')

In [14]:
tp = np.array(list(pix_set.tracking_points))
center = np.array(still_image.shape[:2])/2
radius = 0.75*np.linalg.norm(center)
distances = np.linalg.norm(tp - center, axis=1)
inside_web = (distances < radius)
lower_section = (tp[:, 0] > 0.1*still_image.shape[0])
tp_lim = tp[inside_web & lower_section]

fig, ax = plt.subplots(figsize=(10, 10))

# plot tp
plt.scatter(center[1], center[0])
plt.scatter(tp[:, 1], tp[:, 0], c='red', s=10, label='tp')

# plot tp_lim with markers on top of tp
plt.scatter(tp_lim[:, 1], tp_lim[:, 0], c='green', s=5, label='tp_lim')
ax.set_xlim([0, still_image.shape[1]])
ax.set_ylim([still_image.shape[0] , 0])
ax.axis('equal')
# show the plot
plt.show()


In [ ]:
path = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'
with open(path + file_name+'.pkl', 'wb') as f:
    for key, value in pix_set.__dict__.items():
        if key == 'image':
            value = np.array(value)
        try:
            dill.dump((key, value), f)
        except:
            print(f'Could not pickle {key}')

In [15]:
video.set_points(np.array([pix_set.reference_image_center_i,pix_set.reference_image_center_j]).T)
video.set_method('lk')
# video.method.configure(mraw_range = (sequential_image_n, video.N))
video.show_points()
video.method.resume_analysis = False


In [16]:
d = video.get_displacements()
# video.method.valid_points
d = video.displacements
d


--- Starting new analysis ---
 
Interpolating the reference image...
...done in 0.00 s


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\ipywidgets\widgets\widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


VBox()

Time to complete: 1 min, 27.7 s
D:/thijsmas/HSC\ecc0\web1\Full_web_ecc0_new2_Floc15_v0_S01\Full_web_ecc0_new2_Floc15_v0_S01_pyidi_analysis\analysis_001\ analysis_001


array([[[ 0.00000000e+00,  0.00000000e+00],
        [ 1.57341630e-02, -4.07092768e-04],
        [-1.68544107e-03, -7.92285326e-04],
        ...,
        [ 2.37994981e+00,  3.41403277e-01],
        [ 2.37571339e+00,  3.43873382e-01],
        [ 2.38761289e+00,  3.39878324e-01]],

       [[ 0.00000000e+00,  0.00000000e+00],
        [ 3.22691703e-03,  9.02827171e-03],
        [ 1.48516906e-02,  6.89043762e-02],
        ...,
        [-3.89720998e+00, -2.90051969e+01],
        [-4.05035483e+00, -2.97328756e+01],
        [-3.86578878e+00, -2.89132400e+01]],

       [[ 0.00000000e+00,  0.00000000e+00],
        [ 4.71965882e-04,  2.93001992e-03],
        [ 1.65251989e-03,  5.79384031e-04],
        ...,
        [ 1.29446790e+00,  3.10796494e-02],
        [ 1.30758856e+00,  3.73242235e-02],
        [ 1.31245229e+00,  3.96710564e-02]],

       [[ 0.00000000e+00,  0.00000000e+00],
        [ 7.43709781e-03,  2.77057650e-04],
        [ 1.89969161e-02, -3.84117702e-04],
        ...,
        [ 1.248034

In [ ]:
import pickle
tp_lim = np.array([pix_set.reference_image_center_i,pix_set.reference_image_center_j]).T
root = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'
path = os.path.join(root, file_name+'_d'+'.pkl')
with open(path, 'wb') as f:
    pickle.dump({'displacement':d, 'tracking points': tp_lim}, f)


In [28]:
import importlib
import pixel_setter
importlib.reload(pixel_setter)
from pixel_setter import play_video


# tp = np.array(list(pix_set.tracking_points))
td = d +  np.array([pix_set.reference_image_center_i,pix_set.reference_image_center_j]).T.reshape(len(tp_lim)+1,1,2)
ani = play_video(video, range(sequential_image_n+800, video.N), interval=30, points=td, show_saturation=True) #
# writer = animation.FFMpegWriter(fps=60)
# ani.save('G:\\.shortcut-targets-by-id\\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\\Illimited Lab Projects\\Spiders\\Simulations\\LK demo\\' +file_name+ '_anim2.mp4', writer=writer)

Mask points with high amplitude, outside of web, and irregular vibration

In [ ]:
# center = np.array(still_image.shape[:2])/2
# radius = 0.75*np.linalg.norm(center)
# distances = np.linalg.norm(tp - center, axis=1)
# inside_web = (distances < radius) #& tp[0,:] > still_image.shape[0]*.1

low_amplitude = np.max(np.linalg.norm(d, axis=2),1)<50
ani = play_video(video, range(500,len(pix_set.tracking_points)), interval=30, points=td[low_amplitude,:,:])

Find the gradient off the intensity accross the still figure

In [ ]:
gy, gx = np.gradient(still_image, edge_order=2)
g_norm = np.sqrt(gx**2 + gy**2)
arg_g  = g_norm > 15000
pix_gi, pix_gj = np.where(arg_g)
fig, ax = plt.subplots()
ax.imshow(still_image, cmap='gray')
ax.plot(pix_gj, pix_gi, 'r.')

In [ ]:
pix_set.generate_reference_image(plot=True, seperate_save=True)

In [ ]:
sliders = pix_set.cross_correlate_all()

In [ ]:
# white_value, black_value = pix_set.get_white_black_value()
# pix_set.plot_histogram(bins=50)

# fig, ax = plt.subplots()

# colors = [(1, 0, 0), (0, 1, 0)]
# markers = ['o', 's', 'v', '^', 'D', 'p', '*', 'X', 'd', 'h']
# markersize = 10
# cmap = mcolors.LinearSegmentedColormap.from_list('red_to_green', colors, N=256)

# still_image_temp = still_image.copy() - still_image.mean()
# Low_I = still_image < 20000
# # reference_crosses.append(still_image_temp[263:270, 334:341].astype(np.float64))
# corr_max = np.zeros((still_image.shape[0], still_image.shape[1]))
# for i, reference_cross in enumerate(pix_set.ref_imgs):
#     # template = reference_crosses[0].astype(np.float64)
#     # template -= template.mean()
#     # corr = correlate2d(still_image_temp, template, boundary='symm', mode='same')
#     # corr = match_template(still_image, reference_cross, pad_input=True)
#     # high_corr = corr > 0.75
#     # corr = (corr-np.min(corr)) / np.max(corr-np.min(corr))
#     # peaks_local = detect_peaks(corr)
#     # y, x = np.where(peaks_local & high_corr & Low_I) # & high_corr
#     x, y, corr = pix_set.cross_correlate(reference_cross)
#     ax.scatter(x, y, marker=markers[i % len(markers)], s=markersize*3)
#     markersize -= 1
#     corr_max = np.maximum(corr_max, corr)



ax.imshow(still_image, cmap='gray')
num_plots = len(pix_set.ref_imgs)
if num_plots == 1:
    fig, ax = plt.subplots()
    ax.imshow(pix_set.ref_imgs[0], cmap='gray')
elif num_plots == 2:
    fig, ax = plt.subplots(ncols=2)
    for i, ref_cross in enumerate(pix_set.ref_imgs):
        ax[i].imshow(ref_cross, cmap='gray')
else:
    num_cols = np.ceil(np.sqrt(num_plots))
    num_rows, remainder = divmod(num_plots, num_cols)
    if remainder > 0:
        num_rows += 1

    fig, ax = plt.subplots(nrows = int(num_rows), ncols= int(num_cols))
    for i, ref_cross in enumerate(pix_set.ref_imgs):
        row = int(i // num_cols)
        col = int(i % num_cols)
        ax[row, col].imshow(ref_cross, cmap='gray')
plt.show()


In [ ]:
template = still_image[263:270, 334:341]
result = match_template(still_image, template)
ij = np.unravel_index(np.argmax(result), result.shape)
x, y = ij[::-1]
fig, ax = plt.subplots()
ax.imshow(still_image, cmap='gray')
hcoin, wcoin = template.shape
rect = plt.Rectangle((x, y), wcoin, hcoin, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()

Set limits on the Pixel intensity, Gradient x, and Gradient y to define a Region of Interest (ROI)
By drawing polygons on the figure and pressing enter, parts of the ROI can be removed.

In [ ]:
%matplotlib qt
arg_pi = still_image < 70000
arg_gy = np.linalg.norm([gy, gx], axis=0) > 28000
args = arg_pi & arg_gy #| arg_gx
pix_i, pix_j = np.where(args)


pixelsetter = PixelSetter(still_image, pix_i, pix_j, file_name, sequential_image_n)
pixelsetter.set_method('by_hand')
pixelsetter.get_tracking_pixels()


In [ ]:
pixelsetter.plot_histogram(inside_polygon=True)
pixelsetter.save()

In [ ]:
# del pixelsetter
pixelsetter = PixelSetter.load(file_name)
sequential_image_n = pixelsetter.frame_nr
pix_i = pixelsetter.pix_i
pix_j = pixelsetter.pix_j

Since the above process can be time consuming, the code below can be used to replot the ROI

In [ ]:
# pix_i, pix_j, np.linalg.norm([gy, gx], axis=0).max()

In [ ]:
%matplotlib qt
# mraw_range = (sequential_image_n, sequential_image_n+800) 
# n_frames = mraw_range[1] - mraw_range[0]

# fig, ax = plt.subplots(figsize=(15, 5))
# ax.imshow(still_image, cmap='gray')
# plt.plot(pix_j, pix_i, 'r.', markersize=1)
video.set_points(np.stack((pix_i, pix_j), axis=-1))
video.set_method('lk')
video.method.configure()  #mraw_range=(sequential_image_n, video.N) reference_range=(1,2)
# print((video.method.start_time, video.method.stop_time))
video.show_points()


In [ ]:
# %matplotlib inline
# ref_img = video.method.reference(video.mraw[video.method.reference_range[0]: video.method.reference_range[1]], video.method.subset_size)
# fig, ax = plt.subplots()
# ax.imshow(still_image, cmap='gray')
# plt.show()

In [ ]:
d = video.get_displacements()
# video.method.valid_points
d = video.displacements
d

In [31]:
%matplotlib qt
fig, ax = plt.subplots()
col = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
labels = ['Little contrast', 'Edge', 'Intersection', 'Undersaturated', 'Oversaturated']
for point in np.arange(d.shape[0]): #[video.method.valid_points]
    ax.plot(np.arange(sequential_image_n, video.N), d[point, :, 0],col[point], label=labels[point])#+' - y') #video.info['Record Rate(fps)']*
    # ax.plot(np.arange(sequential_image_n, video.N), d[point, :, 1],col[point] + '--') #video.info['Record Rate(fps)']* , label=labels[point]+' - x'
ax.legend()
# ax.set_ylim(-50, 50)
ax.set_xlabel('frame number')
ax.set_ylabel('y displacement [pixels]')
ax.set_xlim(500, 1200)

(500.0, 1200.0)

In [ ]:
# Add pix_i and pix_j to displacement to get the tracked position
# mask = np.any(d > 3, axis=(1,2))
tp = (np.stack((pix_i, pix_j), axis=-1)).reshape(len(pix_i),1,2) + d # d[:,:n_frames,:]
# tp[mask, :, :] = np.nan
ani = play_video(video, range(1,799), interval=60, points=tp ) #axis=([720,760],[300,230])axis = ([450,350],[460,560])
# ani = play_video(video, range(10000), interval=30)
# writer = animation.FFMpegWriter(fps=60)
# path = 'C:/Users/thijs/Documents/GitHub/pyidi_data/anim/'
# ani.save(path+file_name + '_LK.mp4', writer=writer)

In [ ]:
import os

folder_path = 'H:/My Drive/PHD/HSC'
all_files_drive = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_drive.append(file)

folder_path = 'D:/HSC'
all_files_T7 = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_T7.append(file)

folder_path = 'F:/'
all_files_Sandisk = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_Sandisk.append(file)


folder_path = 'E:/thijs'
all_files_Passport = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_Passport.append(file)

folder_path = 'C:/Users/thijs/Documents/HSC'
all_files_internal = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_internal.append(file)


In [ ]:
unique_drive_files = set(all_files_drive) - set(all_files_T7) - set(all_files_Sandisk) - set(all_files_Passport) - set(all_files_internal)
unique_drive_files